In [1]:
import pandas as pd
import numpy as np
import os, random, math, glob
from IPython.display import Image as IM
from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]

In [98]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submission.csv')

In [99]:
train.head()

,Won_Championship,Previous_SB_Wins,Number_Of_Wins_This_Season,Number_Of_First_Round_Draft_Picks,Team_Value,Playing_Style,Average_Player_Age,Number_Of_Injured_Players,Coach_Experience_Level,ID
0,0,3,13,2,Less_Than_Four_Billion,Balanced,27,five,Intermediate,6056
1,0,2,14,2,Less_Than_Four_Billion,Aggressive_Offense,26,five,Intermediate,9702
2,1,2,13,1,Less_Than_Four_Billion,Aggressive_Defense,27,four,Intermediate,1745
3,0,2,12,2,Above_Four_Billion,Balanced,27,six,Intermediate,4757
4,0,1,15,2,Less_Than_Four_Billion,Balanced,26,five,Intermediate,7242


In [100]:
train['Won_Championship'].value_counts()

1    3255
0    3245
Name: Won_Championship, dtype: int64

In [101]:
test_id = test['ID']

In [102]:
test = test.drop(['ID', 'Playing_Style'], axis=1)

In [103]:
train = train.drop(['ID', 'Playing_Style'], axis=1)

In [76]:
train.head()

,Won_Championship,Previous_SB_Wins,Number_Of_Wins_This_Season,Number_Of_First_Round_Draft_Picks,Team_Value,Average_Player_Age,Number_Of_Injured_Players,Coach_Experience_Level
0,0,3,13,2,Less_Than_Four_Billion,27,five,Intermediate
1,0,2,14,2,Less_Than_Four_Billion,26,five,Intermediate
2,1,2,13,1,Less_Than_Four_Billion,27,four,Intermediate
3,0,2,12,2,Above_Four_Billion,27,six,Intermediate
4,0,1,15,2,Less_Than_Four_Billion,26,five,Intermediate


In [79]:
#train['level+Value'] = train['Coach_Experience_Level'] + '_' + train['Team_Value']
#test['level+Value'] = test['Coach_Experience_Level'] + '_' + test['Team_Value']

In [104]:
X = train.drop('Won_Championship', axis=1)
y = train['Won_Championship']

In [105]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [106]:
from sklearn.model_selection import train_test_split, StratifiedKFold

In [107]:
# Split the data into 30% validation and 70% training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1234)

In [108]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [109]:
model = CatBoostClassifier(#reg_lambda=5,#learning_rate=0.1,#200 best so far
                          loss_function='Logloss', eval_metric='F1',
                           #learning_rate=0.03,
                       metric_period=20)#5

In [110]:
#model.fit(X_train, y_train, cat_features=categorical_features_indices)

In [111]:
from sklearn.metrics import accuracy_score,confusion_matrix,roc_auc_score, mean_absolute_error, mean_squared_error
errcb=[]
y_pred_totcb=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit, GroupKFold
fold=StratifiedKFold(n_splits=40,shuffle=True, random_state=101)#101
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index],
    y_train, y_test = y[train_index], y[test_index]
    m=model#LGBMRegressor(**params)
    #LGBMRegressor(**params)
    m.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)],cat_features=categorical_features_indices, early_stopping_rounds=100)
    preds=m.predict(X_test)
    print("err: ",accuracy_score(y_test,preds))
    errcb.append(accuracy_score(y_test,preds))
    p = m.predict(test)
    y_pred_totcb.append(p)

Learning rate set to 0.09472
0:	learn: 0.5420254	test: 0.5411862	test1: 0.5123967	best: 0.5123967 (0)	total: 15.2ms	remaining: 15.2s


20:	learn: 0.7514879	test: 0.7614358	test1: 0.7672956	best: 0.7701863 (10)	total: 213ms	remaining: 9.91s
40:	learn: 0.7611988	test: 0.7697463	test1: 0.7701863	best: 0.7721519 (21)	total: 478ms	remaining: 11.2s
60:	learn: 0.7658004	test: 0.7704082	test1: 0.7777778	best: 0.7777778 (59)	total: 705ms	remaining: 10.8s
80:	learn: 0.7702162	test: 0.7737226	test1: 0.7926829	best: 0.7926829 (74)	total: 965ms	remaining: 10.9s
100:	learn: 0.7719974	test: 0.7750040	test1: 0.7926829	best: 0.7926829 (74)	total: 1.21s	remaining: 10.8s
120:	learn: 0.7761051	test: 0.7751348	test1: 0.7926829	best: 0.7926829 (74)	total: 1.49s	remaining: 10.8s
140:	learn: 0.7773150	test: 0.7743166	test1: 0.7926829	best: 0.7926829 (74)	total: 1.78s	remaining: 10.9s
160:	learn: 0.7799648	test: 0.7748092	test1: 0.7852761	best: 0.7926829 (74)	total: 2.06s	remaining: 10.7s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7926829268
bestIteration = 74

Shrink model to first 75 iterations.
err:  0.79268292682

20:	learn: 0.7572199	test: 0.7671534	test1: 0.7261146	best: 0.7261146 (16)	total: 217ms	remaining: 10.1s
40:	learn: 0.7645740	test: 0.7719911	test1: 0.7320261	best: 0.7320261 (23)	total: 428ms	remaining: 10s
60:	learn: 0.7679487	test: 0.7736417	test1: 0.7320261	best: 0.7320261 (23)	total: 651ms	remaining: 10s
80:	learn: 0.7702897	test: 0.7747235	test1: 0.7320261	best: 0.7320261 (23)	total: 885ms	remaining: 10s
100:	learn: 0.7749242	test: 0.7774976	test1: 0.7354839	best: 0.7354839 (99)	total: 1.13s	remaining: 10.1s
120:	learn: 0.7784850	test: 0.7777603	test1: 0.7354839	best: 0.7354839 (99)	total: 1.41s	remaining: 10.3s
140:	learn: 0.7802005	test: 0.7767266	test1: 0.7354839	best: 0.7354839 (99)	total: 1.7s	remaining: 10.3s
160:	learn: 0.7829544	test: 0.7807086	test1: 0.7225806	best: 0.7354839 (99)	total: 2s	remaining: 10.4s
180:	learn: 0.7868801	test: 0.7814838	test1: 0.7225806	best: 0.7354839 (99)	total: 2.39s	remaining: 10.8s
Stopped by overfitting detector  (100 iterations wait)

best

20:	learn: 0.7557972	test: 0.7694958	test1: 0.7836257	best: 0.7836257 (18)	total: 289ms	remaining: 13.5s
40:	learn: 0.7622108	test: 0.7742540	test1: 0.7790698	best: 0.7836257 (18)	total: 573ms	remaining: 13.4s
60:	learn: 0.7652620	test: 0.7747184	test1: 0.7790698	best: 0.7836257 (18)	total: 761ms	remaining: 11.7s
80:	learn: 0.7681484	test: 0.7729926	test1: 0.7882353	best: 0.7882353 (65)	total: 974ms	remaining: 11s
100:	learn: 0.7717565	test: 0.7781051	test1: 0.7976879	best: 0.7976879 (98)	total: 1.21s	remaining: 10.7s
120:	learn: 0.7756594	test: 0.7775365	test1: 0.7906977	best: 0.7976879 (98)	total: 1.44s	remaining: 10.4s
140:	learn: 0.7775837	test: 0.7771731	test1: 0.7906977	best: 0.7976879 (98)	total: 1.73s	remaining: 10.6s
160:	learn: 0.7792907	test: 0.7776735	test1: 0.7906977	best: 0.7976879 (98)	total: 2.03s	remaining: 10.6s
180:	learn: 0.7811608	test: 0.7781070	test1: 0.7976879	best: 0.7976879 (98)	total: 2.35s	remaining: 10.6s
Stopped by overfitting detector  (100 iterations wai

20:	learn: 0.7574984	test: 0.7677277	test1: 0.7450980	best: 0.7450980 (15)	total: 248ms	remaining: 11.6s
40:	learn: 0.7657125	test: 0.7738411	test1: 0.7402597	best: 0.7450980 (15)	total: 469ms	remaining: 11s
60:	learn: 0.7675052	test: 0.7722804	test1: 0.7368421	best: 0.7450980 (15)	total: 751ms	remaining: 11.6s
80:	learn: 0.7693541	test: 0.7727705	test1: 0.7368421	best: 0.7450980 (15)	total: 1s	remaining: 11.4s
100:	learn: 0.7732865	test: 0.7738869	test1: 0.7368421	best: 0.7450980 (15)	total: 1.26s	remaining: 11.2s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7450980392
bestIteration = 15

Shrink model to first 16 iterations.
err:  0.7621951219512195
Learning rate set to 0.09472
0:	learn: 0.5427524	test: 0.5394601	test1: 0.5806452	best: 0.5806452 (0)	total: 15.7ms	remaining: 15.7s


...

20:	learn: 0.7561758	test: 0.7542839	test1: 0.7455621	best: 0.7529412 (19)	total: 202ms	remaining: 9.4s
40:	learn: 0.7633004	test: 0.7657886	test1: 0.7573964	best: 0.7573964 (37)	total: 421ms	remaining: 9.85s
60:	learn: 0.7666402	test: 0.7727344	test1: 0.7816092	best: 0.7816092 (49)	total: 665ms	remaining: 10.2s
80:	learn: 0.7707006	test: 0.7722267	test1: 0.7682927	best: 0.7836257 (67)	total: 882ms	remaining: 10s
100:	learn: 0.7737319	test: 0.7730879	test1: 0.7764706	best: 0.7836257 (67)	total: 1.12s	remaining: 10s
120:	learn: 0.7758868	test: 0.7741117	test1: 0.7764706	best: 0.7836257 (67)	total: 1.36s	remaining: 9.88s
140:	learn: 0.7802093	test: 0.7776386	test1: 0.7745665	best: 0.7836257 (67)	total: 1.62s	remaining: 9.86s
160:	learn: 0.7814234	test: 0.7774471	test1: 0.7745665	best: 0.7836257 (67)	total: 1.87s	remaining: 9.73s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.783625731
bestIteration = 67

Shrink model to first 68 iterations.
err:  0.7716049382716049


20:	learn: 0.7619348	test: 0.7644331	test1: 0.7333333	best: 0.7435897 (9)	total: 200ms	remaining: 9.31s
40:	learn: 0.7646966	test: 0.7679000	test1: 0.7500000	best: 0.7500000 (21)	total: 389ms	remaining: 9.09s
60:	learn: 0.7693039	test: 0.7714878	test1: 0.7500000	best: 0.7500000 (21)	total: 618ms	remaining: 9.51s
80:	learn: 0.7724750	test: 0.7717565	test1: 0.7500000	best: 0.7581699 (74)	total: 800ms	remaining: 9.08s
100:	learn: 0.7758838	test: 0.7782466	test1: 0.7547170	best: 0.7594937 (93)	total: 1.06s	remaining: 9.4s
120:	learn: 0.7775677	test: 0.7780556	test1: 0.7468354	best: 0.7594937 (93)	total: 1.32s	remaining: 9.61s
140:	learn: 0.7793353	test: 0.7777604	test1: 0.7515924	best: 0.7594937 (93)	total: 1.62s	remaining: 9.85s
160:	learn: 0.7804878	test: 0.7783682	test1: 0.7468354	best: 0.7594937 (93)	total: 1.9s	remaining: 9.92s
180:	learn: 0.7822530	test: 0.7790625	test1: 0.7547170	best: 0.7594937 (93)	total: 2.23s	remaining: 10.1s
200:	learn: 0.7842089	test: 0.7800778	test1: 0.754717

20:	learn: 0.7573123	test: 0.7654282	test1: 0.8363636	best: 0.8363636 (19)	total: 249ms	remaining: 11.6s
40:	learn: 0.7613437	test: 0.7674529	test1: 0.8518519	best: 0.8518519 (40)	total: 447ms	remaining: 10.5s
60:	learn: 0.7662835	test: 0.7671537	test1: 0.8518519	best: 0.8518519 (40)	total: 822ms	remaining: 12.6s
80:	learn: 0.7700398	test: 0.7685215	test1: 0.8447205	best: 0.8518519 (40)	total: 1.06s	remaining: 12s
100:	learn: 0.7723940	test: 0.7699143	test1: 0.8447205	best: 0.8518519 (40)	total: 1.31s	remaining: 11.7s
120:	learn: 0.7752827	test: 0.7705724	test1: 0.8447205	best: 0.8518519 (40)	total: 1.59s	remaining: 11.6s
140:	learn: 0.7798822	test: 0.7706946	test1: 0.8447205	best: 0.8518519 (40)	total: 1.86s	remaining: 11.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8518518519
bestIteration = 40

Shrink model to first 41 iterations.
err:  0.8518518518518519
Learning rate set to 0.094723
0:	learn: 0.5439968	test: 0.5442150	test1: 0.4958678	best: 0.4958678 (0)

20:	learn: 0.7579821	test: 0.7645850	test1: 0.7515924	best: 0.7515924 (8)	total: 218ms	remaining: 10.2s
40:	learn: 0.7649123	test: 0.7684574	test1: 0.7594937	best: 0.7594937 (34)	total: 389ms	remaining: 9.09s
60:	learn: 0.7691819	test: 0.7725711	test1: 0.7654321	best: 0.7654321 (53)	total: 639ms	remaining: 9.83s
80:	learn: 0.7723150	test: 0.7779839	test1: 0.7515152	best: 0.7654321 (53)	total: 873ms	remaining: 9.9s
100:	learn: 0.7765941	test: 0.7796088	test1: 0.7590361	best: 0.7654321 (53)	total: 1.11s	remaining: 9.84s
120:	learn: 0.7773537	test: 0.7791003	test1: 0.7590361	best: 0.7654321 (53)	total: 1.37s	remaining: 9.93s
140:	learn: 0.7788232	test: 0.7807453	test1: 0.7636364	best: 0.7654321 (53)	total: 1.68s	remaining: 10.2s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7654320988
bestIteration = 53

Shrink model to first 54 iterations.
err:  0.7654320987654321
Learning rate set to 0.094723
0:	learn: 0.5403292	test: 0.5408416	test1: 0.5156250	best: 0.5156250 (0)

20:	learn: 0.7549413	test: 0.7659105	test1: 0.7530864	best: 0.7530864 (12)	total: 218ms	remaining: 10.2s
40:	learn: 0.7606620	test: 0.7707516	test1: 0.7530864	best: 0.7577640 (23)	total: 493ms	remaining: 11.5s
60:	learn: 0.7638288	test: 0.7713332	test1: 0.7547170	best: 0.7577640 (23)	total: 770ms	remaining: 11.9s
80:	learn: 0.7658686	test: 0.7705518	test1: 0.7594937	best: 0.7594937 (74)	total: 1s	remaining: 11.3s
100:	learn: 0.7688846	test: 0.7702616	test1: 0.7594937	best: 0.7594937 (74)	total: 1.28s	remaining: 11.4s
120:	learn: 0.7716485	test: 0.7728069	test1: 0.7594937	best: 0.7594937 (74)	total: 1.54s	remaining: 11.2s
140:	learn: 0.7747402	test: 0.7750317	test1: 0.7594937	best: 0.7594937 (74)	total: 1.86s	remaining: 11.3s
160:	learn: 0.7761099	test: 0.7745921	test1: 0.7594937	best: 0.7594937 (74)	total: 2.19s	remaining: 11.4s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.7594936709
bestIteration = 74

Shrink model to first 75 iterations.
err:  0.76543209876543

In [112]:
np.mean(errcb)

0.7781638111626515

In [60]:
#model.fit(X, y,cat_features=categorical_features_indices, early_stopping_rounds=100)

In [61]:
#preds = model.predict(test)

In [62]:
#pd.DataFrame({'ID':test_id, 'Won_Championship':preds.astype(int)}).to_csv('win27.csv', index=False)

In [120]:
preds = np.mean(y_pred_totcb, 0)

In [121]:
pred = pd.DataFrame(preds, columns=['Won_Championship'])

In [122]:
pred['Won_Championship'].value_counts()

0.000    1587
1.000    1405
0.900      94
0.950      86
0.150      33
0.975      29
0.125      27
0.475      25
0.875      23
0.525      20
0.825      16
0.175      15
0.050      15
0.925      14
0.500      10
0.350      10
0.400      10
0.250       9
0.625       9
0.650       7
0.100       7
0.025       6
0.850       5
0.725       5
0.575       5
0.300       4
0.425       3
0.225       3
0.550       3
0.675       3
0.275       2
0.750       2
0.600       2
0.325       1
0.800       1
0.700       1
0.200       1
0.775       1
0.075       1
Name: Won_Championship, dtype: int64

In [123]:
def thresh(x):
    if x <= 0.3:
        return 0
    else:
        return 1

In [124]:
pred['Won_Championship'] = pred['Won_Championship'].apply(thresh)

In [125]:
pred['Won_Championship'].value_counts()

1    1790
0    1710
Name: Won_Championship, dtype: int64

In [126]:
pd.DataFrame({'ID':test_id, 'Won_Championship':pred['Won_Championship'].astype(int)}).to_csv('win38.csv', index=False)

In [ ]:
LGBMClassifier(num_leaves= 5, max_depth=5, 
                         random_state=314,
                         metric='None', 
                         n_jobs=4, class_weight='balanced',
                         n_estimators=3000,
                         #colsample_bytree=0.9,
                         subsample=0.9,
                         learning_rate=0.015)